In [1]:
import icalendar
import pytz, zoneinfo, dateutil.tz  # timezone libraries
import datetime, icalendar
import pyaml
import pandas as pd 
import numpy as np
import yaml
from datetime import datetime, timedelta
import re
from ics import Calendar, Event
import math

In [2]:




def convert_datestring (datestring):
    format = "%m/%d/%Y"
    date_object = datetime.strptime(datestring, format)
    return date_object


def get_start_end_dates(block_id, my_dict):
    start, end = (my_dict['block_dates'][block_id].split())
    start = convert_datestring(start)
    end = convert_datestring(end)
    return start, end


def date_lists(start_date, end_date):
    current_date = start_date
    dates = []
    while current_date <= end_date:
        dates.append(current_date)
        current_date += timedelta(days=1)
    return dates
    

        
def block_dates (block_id, my_dict):
    """get start and end date for each block by going through the master yaml """
    start_date, end_date = (get_start_end_dates(block_id, my_dict))
    block_duration = date_lists(start_date, end_date)    
    return block_duration
 

def add_date_postcall_into_next_block(call_cycle, block_days):
    print(call_cycle[-1])
    if re.search(r'^\*', call_cycle[-1]):
        # Step 1: Get the last date in the list
        last_date = block_days[-1]

        # Step 2: Calculate the next date (assuming consecutive days, so add 1 day)
        next_date = last_date + timedelta(days=1)

        # Step 3: Add the new date to the list
        block_days.append(next_date)
    return block_days
    
    
def get_specific_rotation_schedule(block_id, rotation, my_dict):
    rotation = re.sub('\s', '', rotation)
    

        
    try:
        rotation, individual = rotation.split('-')
    except:
        rotation = rotation
        individual = 'Z'
        
    if re.search('Janeway|Barker|Longcope|Thayer', rotation):
        rotation_category = 'FirmSAR'    
    elif re.search('CJ|Polk|MEG|Liver', rotation):
        rotation_category = 'Subspecialty'    
        
    else:
        rotation_category = rotation
    print(rotation_category)
    print(individual)
    print(my_dict[rotation_category])
    for schedule_type, schedule in my_dict[rotation_category].items():
        print(schedule['blocks'])
        if block_id in schedule['blocks']:
            print(block_id)
            return schedule[individual].split()
    # if no schedule is found return '-' list
    return ['-'] *14
    

In [3]:
# Read master tables

    
with open("/Users/cjyoon/Dropbox/Osler/Osler_shiny/jarsar2025//call_schedule_master.yaml") as f:
    my_dict = yaml.safe_load(f)

    

    
master_schedule = pd.read_excel('/Users/cjyoon/Dropbox/Osler/Osler_shiny/jarsar2025/Resident Schedule 2025-2026.xlsx', sheet_name='Schedule')

# Combine first two columns into a new 'name' column
master_schedule['name'] = master_schedule.iloc[:, 0] + '_' + master_schedule.iloc[:, 1]

# Optional: drop the original first two columns
master_schedule = master_schedule.drop(master_schedule.columns[[0, 1]], axis=1)
master_schedule = master_schedule[['name'] + [col for col in master_schedule.columns if col != 'name']]

resident_names = master_schedule['name'].tolist()
resident_names = [x for x in resident_names if not (isinstance(x, float) and math.isnan(x))]
resident_names

['Abou Diwan_Elizabeth',
 'Aduboffour_Akwasi',
 'Alexander_Mathew',
 'Amundson_Danielle',
 'Ancha_Bhavya',
 'Arahirwa_Victor',
 'Bediako_Hilary',
 'Blazoski_Cameron',
 'Bothwell_James',
 'Broshkevitch_Anna',
 'Bshesh_Khalifa',
 'Bullock_Avery',
 'Buyske_John',
 'Carter_Julia',
 'Cazimir_Catherine',
 'Chabaan_Lara',
 'Chang_Michael',
 'Clinton_Ivor',
 'Connolly_John',
 'Coyne_Liam',
 'Dobbs_John',
 'Dong_Matthew',
 'Emetu_Sophia',
 'Ermolovich_Jake',
 'Fatteh_Maria',
 'Gao_Ruiyi',
 'Gaona_Geyner',
 'Gasperetti_Alessio',
 'Gharios_Charbel',
 'Goplerud_Dana',
 'Goren_Lea',
 'Guo_Matthew',
 'Gupta_Vrinda',
 'Hawkins_Stephanie',
 'Holler_Albert',
 'Hutto_David',
 'Jabboure_Fayez',
 'Jo_Allen',
 'Johnson_Adrienne',
 'Kankaria_Rohan',
 'Karatela_Maham',
 'Kee_Dustin',
 'Kesaf_Ayse',
 'Khunsriraksakul_Chachrit',
 'Kim_Minjoo',
 'Kodali_Sreeja',
 'Kodukula_Sai',
 'Kosoko_Idris',
 'Koul_Kash',
 'Kuzma_Irena',
 'Littman_Emily',
 'Lopez_Christine',
 'Lopez-Silva_Carolina',
 'Lutz_Lydia',
 'Mansour

In [4]:


def update_values_to_uppercase(data):
    """
    Recursively updates all string values in a nested dictionary to uppercase.
    """
    if isinstance(data, dict):
        return {key: update_values_to_uppercase(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [update_values_to_uppercase(item) for item in data]
    elif isinstance(data, str):
        return data.upper()
    else:
        return data


def add_combined_key(data):
    for key, value in data.items():
        if key.endswith('_') and isinstance(value, dict) and 'A' in value and 'B' in value:
            # Create a list of combined A and B values
            combined = [f"{a.strip()}/{b.strip()}" for (a, b) in zip(value['A'].split(), value['B'].split())]
            value['Z'] = ' '.join(combined)
    return data



def oncology_schedule(rotation, date): 
    schedule = 'DAY'
    
    if date.weekday() == 6 and rotation == 'MTL': #off day is Monday
        schedule = 'OFF'
    elif date.weekday() == 5 and rotation in ['Solids', 'Leuks', 'Heme']: #off day is Saturday
        schedule = 'OFF'
    elif date.weekday() in  [5, 6] and  re.search(r'Pathway|Elective|Research|Geri|UH|Addiction|Women|HCH|HIV|Ambulatory', rotation): #ambo off day is Saturday and Sunday
        schedule = 'OFF'
    else:
        pass
    
    return schedule

# bayview schedule to stadard call formats. Changed in 2025-2026
bayview_conversion = dict({'Call': 'CALL', 'Post': 'POST', 'Continuity': 'CONT', 'Off': "OFF"})

# read in bayview schedule and format column to a string form of dates     
bayview_schedule = pd.read_excel('/Users/cjyoon/Dropbox/Osler/Osler_shiny/jarsar2025/bayview_unit_2025.xlsx')
# Convert columns to string format
# Convert columns to datetime where possible
new_columns = []
for col in bayview_schedule.columns:
    try:
        # Attempt to convert to datetime and format as string
        new_columns.append(pd.to_datetime(col).strftime('%Y-%m-%d'))
    except (ValueError, TypeError):
        # If conversion fails, keep the original column name
        new_columns.append(col)

# Assign the new column names
bayview_schedule.columns = new_columns

def get_bayview_schedule(rotation, date, schedule_df):
    """
    Retrieve the schedule for a specific rotation and date.
    
    Args:
    rotation (str): The rotation type, e.g., 'BMICU-1'.
    date (datetime): The date for which to get the schedule.
    schedule_df (pd.DataFrame): The schedule DataFrame.
    
    Returns:
    str: The schedule activity for the given rotation and date.
    """
#     # Convert the date to string format
#     date_str = date.strftime('%Y-%m-%d')
    
#     # Debug: Print column names and check for the date
#     if date_str not in schedule_df.columns:
#         return f"Date {date_str} not found in the schedule.\nAvailable dates: {schedule_df.columns.tolist()}"

    try:
        # Find the schedule for the given rotation and date
        result = schedule_df.loc[schedule_df['Unnamed: 0'] == rotation, date]
        if not result.empty:
            return result.values[0]
        else:
            return f"Rotation {rotation} not found."
    except KeyError:
        return f"Error occurred while looking up the schedule."


updated_dict = dict()
for key, value in my_dict.items():
    updated_val =  add_combined_key(my_dict[key])
    updated_dict.update({key: updated_val})
    
    
updated_dict = update_values_to_uppercase(updated_dict)
my_dict = updated_dict # overwrite after adding the 'Z' keys for blocks which specific resident role is not defined

In [5]:
my_dict

{'block_dates': {'1A': '7/1/2025  7/7/2025',
  '1B': '7/8/2025  7/21/2025',
  '2A': '7/22/2025 8/4/2025',
  '2B': '8/5/2025  8/18/2025',
  '3A': '8/19/2025 9/1/2025',
  '3B': '9/2/2025  9/15/2025',
  '4A': '9/16/2025 9/29/2025',
  '4B': '9/30/2025 10/13/2025',
  '5A': '10/14/2025  10/27/2025',
  '5B': '10/28/2025  11/10/2025',
  '6A': '11/11/2025  11/24/2025',
  '6B': '11/25/2025  12/8/2025',
  '7A': '12/9/2025  12/19/2025',
  'Holiday 1': '12/20/2025 12/26/2025',
  'Holiday 2': '12/27/2025 1/2/2026',
  '7B': '1/3/2026  1/12/2026',
  '8A': '1/13/2026 1/26/2026',
  '8B': '1/27/2026 2/9/2026',
  '9A': '2/10/2026 2/23/2026',
  '9B': '2/24/2026 3/09/2026',
  '10A': '3/10/2026  3/23/2026',
  '10B': '3/24/2026  4/6/2026',
  '11A': '4/7/2026  4/20/2026',
  '11B': '4/21/2026  5/4/2026',
  '12A': '5/5/2026 5/18/2026',
  '12B': '5/19/2026  6/1/2026',
  '13A': '6/2/2026 6/15/2026',
  '13B': '6/16/2026  6/30/2026'},
 'FirmSAR': {'type1': {'blocks': ['1A'],
   'A': 'OLDS  OLDS  BOOK  OLDS  OFF BOOK

In [6]:
def get_schedule(resident_name, master_schedule):
    """gets the specific intern's schedule from the table"""
    filtered_df = master_schedule[master_schedule.iloc[:, 0] == resident_name]
    return (filtered_df.squeeze().to_list()[1:])



In [7]:
master_schedule

,name,1A,1B,2A,2B,3A,3B,4A,4B,5A,...,9A,9B,10A,10B,11A,11B,12A,12B,13A,13B
0,NaN,7/1/25-7/7/25,7/8/25-7/21/25,7/22/25-8/4/25,8/5/25-8/18/25,8/19/25-9/1/25,9/2/25-9/15/25,9/16/25-9/29/25,9/30/25-10/13/25,10/14/25-10/27/25,...,2/10/26-2/23/26,2/24/26-3/9/26,3/10/26-3/23/26,3/24/26-4/6/26,4/7/26-4/20/26,4/21/26-5/4/26,5/5/26-5/18/26,5/19/26-6/1/26,6/2/26-6/15/26,6/16/26-6/30/26
1,Abou Diwan_Elizabeth,Barker - C,MICU - F,Barker - C,Ambulatory,Vacation,Jeopardy,CCU - A,Ambulatory,Barker,...,MICU,Barker,Elective,Ambulatory,Barker,MICU,Leuks,Ambulatory,Vacation,Elective
2,Aduboffour_Akwasi,Elective,Relief,Ambulatory,Barker - C,Elective-A,Vacation,Ambulatory,MICU - C,Jeopardy,...,Elective,MICU,Ambulatory,Wolf,Barker,Vacation,Ambulatory,Leuks,CCU,Elective
3,Alexander_Mathew,Cardiomyopathy - 4,Polk,Ambulatory,Brancati - C,Firm JAR - D,Pathway-QI,Ambulatory,CJ,Heme,...,BMICU,Leuks,Ambulatory,MPC,Vacation,Brancati,Ambulatory,Elective,Elective,MTL
4,Amundson_Danielle,Peds,Peds,Peds,MP Clinic,Barker - A,Jeopardy,CCU - B,MP Clinic,Peds,...,Geri,Geri,Peds,Peds,Peds,Peds,Vacation,MP Clinic,UH Procedure,Barker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Zale_Andrew,MICU - C,Longcope - C,Ambulatory,Triage,Jeopardy,Longcope - A,Ambulatory,Vacation,Wolf,...,NATO/DATO,Elective,Ambulatory,Vacation,Polk,Jeopardy,Ambulatory,Longcope,MICU,Elective
101,Zarker_Andrew,Ambulatory,Cardiomyopathy - 1,Jeopardy,Liver,Ambulatory,Firm JAR - C,Leuks,Vacation,Ambulatory,...,Ambulatory,Elective,Polk,Elective,Vacation,Ambulatory,BMICU,Solids,Ambulatory,Gen Cards
102,Zeitoun_Ralph,Janeway-C,CCU - C,Janeway - C,Ambulatory,Vacation,Elective,MICU - A,Ambulatory,Janeway,...,Triage,MICU,Elective,Ambulatory,Janeway,Relief,Jeopardy,Ambulatory,Vacation,Elective
103,Zerjav_Dylan,Ambulatory,CCU - B,Night Watch,Longcope - A,Ambulatory,MICU - F,Longcope,Elective,Ambulatory,...,Ambulatory,Elective,Triage,CCU,Ambulatory,Longcope,MICU,Vacation,Ambulatory,Elective


In [8]:
print(master_schedule)

header_row = master_schedule[master_schedule['name'].isna()].iloc[0]
print(header_row)
# Step 2: Create new column names by combining old column name + row value
new_columns = [f"{col} {header_row[col]}" if col != 'name' else 'name' for col in master_schedule.columns]

# Step 3: Remove the row with NaN in 'name'
master_schedule = master_schedule[master_schedule['name'].notna()].copy()

# Step 4: Assign new column names
master_schedule.columns = new_columns

print(master_schedule)


master_schedule.replace("Supervisor", "Wolf", inplace=True)

master_schedule.sort_values('name').to_csv('/Users/cjyoon/Dropbox/Osler/Osler_shiny/jarsar2025/block_view_jarsar.tsv', sep='\t', index=False)

                     name                   1A                  1B  \
0                     NaN        7/1/25-7/7/25      7/8/25-7/21/25   
1    Abou Diwan_Elizabeth           Barker - C            MICU - F   
2       Aduboffour_Akwasi             Elective              Relief   
3        Alexander_Mathew  Cardiomyopathy - 4                 Polk   
4       Amundson_Danielle                 Peds                Peds   
..                    ...                  ...                 ...   
100           Zale_Andrew             MICU - C        Longcope - C   
101         Zarker_Andrew           Ambulatory  Cardiomyopathy - 1   
102         Zeitoun_Ralph            Janeway-C             CCU - C   
103          Zerjav_Dylan           Ambulatory             CCU - B   
104          Ziogos_Timos           Ambulatory  Cardiomyopathy - 3   

                 2A              2B              3A              3B  \
0    7/22/25-8/4/25  8/5/25-8/18/25  8/19/25-9/1/25  9/2/25-9/15/25   
1        Barker -

In [9]:
#july1_offset = my_dict['july1_offset']
# Create a calendar and event


for resident_name in resident_names:
    date_list = []
    schedule_list = []        
    my_blocks = (get_schedule(resident_name, master_schedule))
    print(my_blocks)
    

    calendar = Calendar()
    event = Event()
    with open(f"/Users/cjyoon/Dropbox/Osler/Osler_shiny/jarsar2025/schedule/{resident_name}_schedule.tsv", "w") as g:
        g.write('name\tdate\tschedule\n')
        for block_id, rotation in zip(my_dict['block_dates'].keys(), my_blocks):
            if 1:
                if not isinstance(rotation, str):
                    rotation = ' '

                rotation = re.sub('\s', '', rotation)

                print(block_id, rotation)
                
                block_days = block_dates(block_id, my_dict)        
                print(block_days)
                try:
                    call_cycle = get_specific_rotation_schedule(block_id.upper(), rotation, my_dict)
                except:
                    # if specific call cycle is unknown for a rotation, then default to unknown so manual creation can be made
                    if rotation == ' ':
                        call_cycle = [' '] * 14
                    else:
                        call_cycle = ['-'] * 14



                # check if block call cycle ends in a post call
                block_days = add_date_postcall_into_next_block(call_cycle, block_days)

                # Loop through each date and corresponding schedule
                for date, schedule in zip(block_days, call_cycle):
                    
                    # For Bayview units
                    if re.search(r'BCCU|BMICU', rotation):
                        if rotation=='BCCU':
                            rotation='BCCU-1'
                        elif rotation=="BMICU":
                            rotation='BMICU-1'
                        schedule = bayview_conversion[get_bayview_schedule(rotation, date.strftime("%Y-%m-%d"), bayview_schedule)]
                        print(f'{rotation} bla {schedule}')
                    # For oncology rotations/ambo where schedule is only per weekdays basis
                    elif rotation in ['MTL', 'Leuks', 'Solids', 'Ambulatory', 'Subspecialty', 'Psych', 'UCM', 'AddictionMedicine', 'Geri', 'Elective', 'Heme']:                       
                        schedule = oncology_schedule(rotation, date)
                    elif re.search(r'Pathway|Women|HIV|HCH', rotation):
                        schedule=oncology_schedule(rotation, date)
                        
                        
                    else:
                        pass
                    
                    
                    event = Event()  # Create a new event object inside the loop
                    if rotation == 'Supervisor': # change supervisor into more commonly referred Wolf
                        rotation = 'Wolf'
                    event.name = f'{block_id} {rotation} {schedule}'
                    event.begin = date
                    event.make_all_day()  # Make the event an all-day event
                    print(date, block_id, rotation, schedule)
                    # Add event to calendar
                    calendar.events.add(event)
                    date_string = date.strftime('%m-%d-%Y')
                    g.write(f'{resident_name}\t{date_string}\t{event.name}\n')



        # Write the calendar to a .ics file
        with open(f"/Users/cjyoon/Dropbox/Osler/Osler_shiny/jarsar2025/schedule/{resident_name}_schedule.ics", "w") as f:
            f.writelines(calendar.serialize_iter())

    
    

['Barker - C', 'MICU - F', 'Barker - C', 'Ambulatory', 'Vacation', 'Jeopardy', 'CCU - A', 'Ambulatory', 'Barker', 'Night Watch', 'Jeopardy', 'Ambulatory', 'Elective', 'Off', 'Wolf', 'NATO/DATO', 'Elective', 'Ambulatory', 'MICU', 'Barker', 'Elective', 'Ambulatory', 'Barker', 'MICU', 'Leuks', 'Ambulatory', 'Vacation', 'Elective']
1A Barker-C
[datetime.datetime(2025, 7, 1, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 4, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), datetime.datetime(2025, 7, 7, 0, 0)]
FirmSAR
C
{'type1': {'blocks': ['1A'], 'A': 'OLDS  OLDS  BOOK  OLDS  OFF BOOK  NEWS', 'B': 'NEWS  NEWS  OFF NIGHTS  NIGHTS  NIGHTS  NIGHTS  *POST', 'C': 'NIGHTS  NIGHTS  NIGHTS  NEWS*  BOOK  OFF OLDS'}, 'type2': {'blocks': ['1B', '2A', '2B', '3A', '3B'], 'A': 'NEWS  NEWS  OLDS  OLDS  BOOK  BOOK  NIGHTS NIGHTS NIGHTS  NIGHTS  NEWS* OFF OFF NEWS', 'B': 'NIGHTS  NIGHTS  NIGHTS  NIGHTS  NIGHTS  NIGH

2026-06-29 00:00:00 13B Jeopardy JEOPARDY
['Vacation', 'Ambulatory', 'CCU - C', 'Thayer - A', 'Heme', 'Ambulatory', 'MICU - E', 'Elective', 'Thayer', 'Ambulatory', 'CCU - B', 'Elective', 'Jeopardy', 'Off', 'Jeopardy', 'Ambulatory', 'Thayer', 'Elective', 'Elective', 'Ambulatory', 'Elective', 'Relief', 'Vacation', 'Ambulatory', 'MICU', 'NATO/DATO', 'Thayer', 'Ambulatory']
1A Vacation
[datetime.datetime(2025, 7, 1, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 4, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), datetime.datetime(2025, 7, 7, 0, 0)]
Vacation
Z
{'type1': {'blocks': ['1A', '2A', '3A', '4A', '5A', '6A', '7A', '8A', '9A', '10A', '11A', '12A', '13A', '1B', '2B', '3B', '4B', '5B', '6B', '7B', '8B', '9B', '10B', '11B', '12B', '13B', 'HOLIDAY 1', 'HOLIDAY 2'], 'Z': 'VACATION VACATION VACATION VACATION VACATION VACATION VACATION VACATION VACATION VACATION VACATION VACATION VACATION VACATION

2026-01-05 00:00:00 7B Ambulatory DAY
2026-01-06 00:00:00 7B Ambulatory DAY
2026-01-07 00:00:00 7B Ambulatory DAY
2026-01-08 00:00:00 7B Ambulatory DAY
2026-01-09 00:00:00 7B Ambulatory DAY
2026-01-10 00:00:00 7B Ambulatory OFF
2026-01-11 00:00:00 7B Ambulatory OFF
2026-01-12 00:00:00 7B Ambulatory DAY
8A Thayer
[datetime.datetime(2026, 1, 13, 0, 0), datetime.datetime(2026, 1, 14, 0, 0), datetime.datetime(2026, 1, 15, 0, 0), datetime.datetime(2026, 1, 16, 0, 0), datetime.datetime(2026, 1, 17, 0, 0), datetime.datetime(2026, 1, 18, 0, 0), datetime.datetime(2026, 1, 19, 0, 0), datetime.datetime(2026, 1, 20, 0, 0), datetime.datetime(2026, 1, 21, 0, 0), datetime.datetime(2026, 1, 22, 0, 0), datetime.datetime(2026, 1, 23, 0, 0), datetime.datetime(2026, 1, 24, 0, 0), datetime.datetime(2026, 1, 25, 0, 0), datetime.datetime(2026, 1, 26, 0, 0)]
FirmSAR
Z
{'type1': {'blocks': ['1A'], 'A': 'OLDS  OLDS  BOOK  OLDS  OFF BOOK  NEWS', 'B': 'NEWS  NEWS  OFF NIGHTS  NIGHTS  NIGHTS  NIGHTS  *POST', 'C': 

['MICU - A', 'Ambulatory', 'Janeway - B', 'Vacation', 'MICU - E', 'Ambulatory', 'Janeway', 'Wolf', 'Elective', 'Ambulatory', 'Janeway', 'Research', 'Jeopardy', 'Off', 'Triage', 'Ambulatory', 'Correctional Medicine', 'Janeway', 'Vacation', 'Ambulatory', 'City HD', 'City HD', 'NATO/DATO', 'Ambulatory', 'MICU', 'Janeway', 'Jeopardy', 'Ambulatory']
1A MICU-A
[datetime.datetime(2025, 7, 1, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 4, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), datetime.datetime(2025, 7, 7, 0, 0)]
MICU
A
{'type1': {'blocks': ['1A'], 'A': 'CALL  POST  GOOD  CALL  POST  OFF CALL  POST', 'B': 'OFF CALL  POST  GOOD  CALL  POST  OFF CALL  *POST', 'C': 'POST  GOOD  CALL  POST  OFF CALL  POST  GOOD', 'D': 'CALL  POST  OFF CALL  POST  GOOD  CALL  POST', 'E': 'GOOD  CALL  POST  OFF CALL  POST  GOOD  CALL  *POST', 'F': 'POST  OFF CALL  POST  GOOD  CALL  POST  OFF'}, 'type2a': {'block

['MPC', 'Firm JAR - C', 'Ambulatory', 'Brancati - B', 'BMICU - 1', 'MPC/NW JAR', 'Ambulatory', 'Cardiomyopathy - 1', 'Vacation', 'BCCU - 3', 'Ambulatory', 'Liver', 'Gen Cards', 'Gen Cards', 'Off', 'Elective', 'Ambulatory', 'Cardiomyopathy', 'CJ', 'Elective', 'Ambulatory', 'CCU', 'Elective', 'Vacation', 'Ambulatory', 'Polk', 'Cardiomyopathy', 'MICU']
1A MPC
[datetime.datetime(2025, 7, 1, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 4, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), datetime.datetime(2025, 7, 7, 0, 0)]
MPC
Z
{'type1': {'blocks': ['1A'], 'Z': 'DAYS  DAYS  OFF DAYS  DAYS  DAYS  DAYS'}, 'type2': {'blocks': ['1B', '2A', '2B', '3A', '3B'], 'Z': 'DAYS  DAYS  DAYS  OFF DAYS  DAYS  DAYS DAYS DAYS  DAYS  OFF DAYS  DAYS  DAYS'}, 'type3': {'blocks': ['4A', '4B', '5A', '5B', '6A', '6B', '8A', '8B', '9A', '9B', '10A', '10B', '11A', '11B', '12A', '12B'], 'Z': 'DAYS  DAYS  DAYS  OFF DAYS  DA

...

2025-07-21 00:00:00 1B BCCU-2 CALL
2A Solids
[datetime.datetime(2025, 7, 22, 0, 0), datetime.datetime(2025, 7, 23, 0, 0), datetime.datetime(2025, 7, 24, 0, 0), datetime.datetime(2025, 7, 25, 0, 0), datetime.datetime(2025, 7, 26, 0, 0), datetime.datetime(2025, 7, 27, 0, 0), datetime.datetime(2025, 7, 28, 0, 0), datetime.datetime(2025, 7, 29, 0, 0), datetime.datetime(2025, 7, 30, 0, 0), datetime.datetime(2025, 7, 31, 0, 0), datetime.datetime(2025, 8, 1, 0, 0), datetime.datetime(2025, 8, 2, 0, 0), datetime.datetime(2025, 8, 3, 0, 0), datetime.datetime(2025, 8, 4, 0, 0)]
Solids
Z
{'type1': {'blocks': ['1A', '2A', '3A', '4A', '5A', '6A', '7A', '8A', '9A', '10A', '11A', '12A', '13A', '1B', '2B', '3B', '4B', '5B', '6B', '7B', '8B', '9B', '10B', '11B', '12B', '13B', 'HOLIDAY 1', 'HOLIDAY 2'], False: 'SATURDAY'}}
['1A', '2A', '3A', '4A', '5A', '6A', '7A', '8A', '9A', '10A', '11A', '12A', '13A', '1B', '2B', '3B', '4B', '5B', '6B', '7B', '8B', '9B', '10B', '11B', '12B', '13B', 'HOLIDAY 1', 'HOLID

['Gen Cards', 'Firm JAR - E', 'Cardiomyopathy - 2', 'Ambulatory', 'Pathway-DataSci', 'Elective', 'Vacation', 'Ambulatory', 'CCU - A', 'Jeopardy', 'Elective', 'Ambulatory', 'MICU - B', 'Off', 'BCCU', 'BCCU', 'NATO/DATO', 'Ambulatory', 'Brancati', 'Vacation', 'Liver', 'Ambulatory', 'CJ', 'Cardiomyopathy', 'CCU', 'Ambulatory', 'Cardiomyopathy', 'Cardiomyopathy']
1A GenCards
[datetime.datetime(2025, 7, 1, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 4, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), datetime.datetime(2025, 7, 7, 0, 0)]
GenCards
Z
{'type1': {'blocks': ['1A'], 'Z': 'DAY DAY OFF DAY DAY DAY DAY'}, 'type2': {'blocks': ['1B', '2A', '3A', '4A', '5A', '6A', '8A', '9A', '10A', '11A', '12A', '13A', '2B', '3B', '4B', '5B', '6B', '8B', '9B', '10B', '11B', '12B', '13B'], 'Z': 'DAY DAY DAY OFF DAY DAY DAY DAY DAY OFF DAY DAY DAY DAY'}, 'type3': {'blocks': ['7A'], 'Z': 'DAY DAY DAY OFF DAY DA

2025-11-16 00:00:00 6A Elective OFF
2025-11-17 00:00:00 6A Elective DAY
2025-11-18 00:00:00 6A Elective DAY
2025-11-19 00:00:00 6A Elective DAY
2025-11-20 00:00:00 6A Elective DAY
2025-11-21 00:00:00 6A Elective DAY
2025-11-22 00:00:00 6A Elective OFF
2025-11-23 00:00:00 6A Elective OFF
2025-11-24 00:00:00 6A Elective DAY
6B Wolf
[datetime.datetime(2025, 11, 25, 0, 0), datetime.datetime(2025, 11, 26, 0, 0), datetime.datetime(2025, 11, 27, 0, 0), datetime.datetime(2025, 11, 28, 0, 0), datetime.datetime(2025, 11, 29, 0, 0), datetime.datetime(2025, 11, 30, 0, 0), datetime.datetime(2025, 12, 1, 0, 0), datetime.datetime(2025, 12, 2, 0, 0), datetime.datetime(2025, 12, 3, 0, 0), datetime.datetime(2025, 12, 4, 0, 0), datetime.datetime(2025, 12, 5, 0, 0), datetime.datetime(2025, 12, 6, 0, 0), datetime.datetime(2025, 12, 7, 0, 0), datetime.datetime(2025, 12, 8, 0, 0)]
Wolf
Z
-
2025-11-25 00:00:00 6B Wolf -
2025-11-26 00:00:00 6B Wolf -
2025-11-27 00:00:00 6B Wolf -
2025-11-28 00:00:00 6B Wolf -


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['Ambulatory', 'Cardiomyopathy - 3', 'MPC/NW JAR', 'Firm JAR - B', 'Ambulatory', 'CCU - C', 'Vacation', 'Cardiomyopathy - 4', 'Ambulatory', 'BMICU - 3', 'Elective', 'Cardiomyopathy -1', 'Ambulatory', 'Heme', 'Off', 'Liver', 'CCU', 'Elective', 'Ambulatory', 'Janeway', 'CCU', 'Vacation', 'Ambulatory', 'MICU', 'Polk', 'Cardiomyopathy', 'Ambulatory', 'Janeway']
1A Ambulatory
[datetime.datetime(2025, 7, 1, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), datetime.datetime(2025, 7, 3, 0, 0), datetime.datetime(2025, 7, 4, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2025, 7, 6, 0, 0), datetime.datetime(2025, 7, 7, 0, 0)]
Ambulatory
Z
-
2025-07-01 00:00:00 1A Ambulatory DAY
2025-07-02 00:00:00 1A Ambulatory DAY
2025-07-03 00:00:00 1A Ambulatory DAY
2025-07-04 00:00:00 1A Ambulatory DAY
2025-07-05 00:00:00 1A Ambulatory OFF
2025-07-06 00:00:00 1A Ambulatory OFF
2025-07-07 00:00:00 1A Ambulatory DAY
1B Cardiomyopathy-3
[datetime.datetime(2025, 7, 8, 0, 0), datetime.datetime(2025, 7, 

In [10]:
a=re.search(r'Ambulatory|Elective|UH|HIV|HCH', rotation)

print(a)

None


In [11]:
rotation

'Janeway'

In [12]:
get_specific_rotation_schedule('1A', 'CJ', my_dict)

Subspecialty
Z
{'type1_': {'blocks': ['1A'], 'A': 'DAYS  DAYS  DAYS  DAYS  BOOK  OFF DAYS', 'B': 'DAYS  DAYS  DAYS  DAYS  OFF BOOK  DAYS', 'Z': 'DAYS/DAYS DAYS/DAYS DAYS/DAYS DAYS/DAYS BOOK/OFF OFF/BOOK DAYS/DAYS'}, 'type2_': {'blocks': ['1B', '2A', '2B', '3A', '3B', '4A', '4B', '5A', '5B', '6A', '6B', '8A', '8B', '9A', '9B', '10A', '10B', '11A', '11B', '12A', '12B', '13A', '13B'], 'A': 'DAYS  DAYS  DAYS  DAYS  BOOK  BOOK  DAYS DAYS DAYS  DAYS  DAYS  OFF OFF DAYS', 'B': 'DAYS  DAYS  DAYS  DAYS  OFF OFF DAYS DAYS DAYS  DAYS  DAYS  BOOK  BOOK  DAYS', 'Z': 'DAYS/DAYS DAYS/DAYS DAYS/DAYS DAYS/DAYS BOOK/OFF BOOK/OFF DAYS/DAYS DAYS/DAYS DAYS/DAYS DAYS/DAYS DAYS/DAYS OFF/BOOK OFF/BOOK DAYS/DAYS'}, 'type3_': {'blocks': ['7A'], 'A': 'DAYS  DAYS  DAYS  DAYS  BOOK  OFF DAYS  DAYS  DAYS  DAYS  DAYS', 'B': 'DAYS  DAYS  DAYS  DAYS  OFF BOOK  DAYS  DAYS  DAYS  DAYS  DAYS', 'Z': 'DAYS/DAYS DAYS/DAYS DAYS/DAYS DAYS/DAYS BOOK/OFF OFF/BOOK DAYS/DAYS DAYS/DAYS DAYS/DAYS DAYS/DAYS DAYS/DAYS'}, 'type4_': {'

['DAYS/DAYS',
 'DAYS/DAYS',
 'DAYS/DAYS',
 'DAYS/DAYS',
 'BOOK/OFF',
 'OFF/BOOK',
 'DAYS/DAYS']